**UNIVERSIDAD NACIONAL DE SAN ANTONIO ABAD DEL CUSCO**
> **CURSO:** Minería de datos

> **ALUMNO:**Vega Centeno Olivera, Ronaldinho

## **1.Instalar bibliotecas**

In [1]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 40 kB/s 
     |████████████████████████████████| 198 kB 52.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=ecbc7732e20c95a5117cfef7bd64dc98d4b3c38e2f23f5f74340ffa84ea9cd4a
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


## **2.Conexion a cluster Spark (SparkContext)**

In [2]:
from pyspark import SparkContext
sc =SparkContext() #Permite la creación y manipulación de RDDs

## **3.Bag of words**

In [129]:
def preprocesar(textRDD):
  """
    Modulo para eliminar lineas en blanco, o doc en blanco y 
    dar estructura a la data

  Parameters
  =========
    `textRDD :` RDD con los documentos
          ej -> [ 'founder attack boss ridiculous comment ad',
                  'evidence factcheckthis',
                  'educate']
  Returns
  =========
    `tupleData :` lista de tuplas (idDoc, Doc)
          ej ->[(5, 'founder attack boss ridiculous comment ad'),
                (6, 'evidence factcheckthis'),
                (7, 'educate')]

  """
  #Eliminar lineas vacias
  textRDD=textRDD.filter(lambda x : x is not '')
  #Crear tuplas para dar forma a la data
  tupleData = []
  i=1
  for line in textRDD.collect():
    tupla=(i,line)
    tupleData.append(tupla)
    i+=1
  return tupleData

In [130]:
def bow(data1,particiones=4):
  """
    a. tokenizamos 
    b. uniformizamos, convertimos a minusculas
    c. agregamos un valor a cada tupla : Ej. -> ((1,'hola'),1)

  Parameters
  =========
    `data1 :` lista de tuplas (id, Doc)
          ej -> [(5, 'founder attack boss ridiculous comment ad'),
                (6, 'evidence factcheckthis'),
                (7, 'educate')]
  Returns
  =========
    `parTokenRDD :` lista de tuplas de tuplas ((idDoc, token), 1)
          ej ->[((1, 'protester'), 1),
                ((1, 'stage'), 1),
                ((1, 'diein'), 1)]

  """
  #Creamos el RDD
  linesRDD=sc.parallelize(data1,particiones)
  parTokenRDD=linesRDD.flatMap(lambda x: [((x[0],i.lower()),1) for i in x[1].split()])
  return parTokenRDD


In [131]:
#Crear el RDD leyendo el archivo .txt
textRDD=sc.textFile('twitts.txt',4)
datos=preprocesar(textRDD)
datos
BoW=bow(datos)
BoW.take(20)

[((1, 'protester'), 1),
 ((1, 'stage'), 1),
 ((1, 'diein'), 1),
 ((1, 'protest'), 1),
 ((1, 'store'), 1),
 ((1, 'nyc'), 1),
 ((1, 'anger'), 1),
 ((1, 'misplace'), 1),
 ((1, 'retweet'), 1),
 ((1, 'agree'), 1),
 ((2, 'apple'), 1),
 ((2, 'intraday'), 1),
 ((2, 'comment'), 1),
 ((2, 'update'), 1),
 ((2, 'range'), 1),
 ((2, 'premium'), 1),
 ((2, 'user'), 1),
 ((2, 'aapl'), 1),
 ((2, 'stockaction'), 1),
 ((2, 'trading'), 1)]

## **3.TF**

In [132]:
def frecuencias(bagOfWordsRDD):
  """
  Calcula la frecuencia de cada termino

  Parameters
  =========
    `bagOfWordsRDD :` lista de tuplas de tuplas ((idDoc, token),1)
          ej ->[((1, 'protester'), 1),
                ((1, 'stage'), 1),
                ((1, 'diein'), 1)]
  Returns
  =========
    `frecPerDocumentRDD :`  lista de tuplas de tuplas ((idDoc, token),frecuencia)
          ej ->[((1, 'protester'), 1),
                ((1, 'stage'), 1),
                ((1, 'diein'), 1)]

  """
  #obtenemos la frecuencia de cada palabra por documento
  frecPerDocumentRDD=bagOfWordsRDD.reduceByKey(lambda x,y:x+y)
  return frecPerDocumentRDD

In [133]:
def TF(bagOfWordsRDD):
  """
  Calcula la frecuencia de cada termino

  Parameters
  =========
    `bagOfWordsRDD :` lista de tuplas de tuplas ((idDoc, token),1)
          ej ->[((1, 'protester'), 1),
                ((1, 'stage'), 1),
                ((1, 'diein'), 1)]
  Returns
  =========
    `tfRDD :`  lista de tuplas de tuplas ('token', ('id',frecuencia))
          ej -> [ ('protester', (1, 1)),
                  ('diein', (1, 1)),
                  ('agree', (1, 1)) ]

  """
  frecuenciasRDD=frecuencias(bagOfWordsRDD)
  # Mapear el RDD para obetener ('token', ('id',frecuencia))
  tfRDD=frecuenciasRDD.map(lambda x: (x[0][1],(x[0][0],x[1])))
  return tfRDD

In [134]:
#Crear el RDD leyendo el archivo .txt
textRDD=sc.textFile('twitts.txt',4)
datos=preprocesar(textRDD)
BoW=bow(datos)
tf=TF(BoW)
tf.take(20)

[('protester', (1, 1)),
 ('diein', (1, 1)),
 ('aapl', (2, 1)),
 ('stockaction', (2, 1)),
 ('iphone', (3, 1)),
 ('shit', (4, 1)),
 ('ridiculous', (5, 1)),
 ('suicide', (8, 1)),
 ('turkey', (8, 1)),
 ('ipod', (9, 1)),
 ('aapl', (10, 1)),
 ('stockaction', (10, 1)),
 ('computer', (11, 1)),
 ('future', (11, 1)),
 ('dear', (13, 1)),
 ('plus', (13, 1)),
 ('lot', (13, 1)),
 ('great', (13, 1)),
 ('achievement', (13, 1)),
 ('fail', (13, 1))]

## **4.IDF**

In [135]:
import math
from pyspark.sql.functions import *

In [136]:
def IDF(bagOfWordsRDD,cantidadDoc):
  """
  Calcula Inverse document frequency
  
  Parameters
  =========
    `bagOfWordsRDD :` lista de tuplas de tuplas ((idDoc, token),1)
          ej ->[((1, 'protester'), 1),
                ((1, 'stage'), 1),
                ((1, 'diein'), 1)]
    
    `cantidadDoc :` cantidad de documentos (int)
          ej -> 2857
    
  Returns
  =========
    `idfRDD :`  lista de tuplas ('token', idf(float))
          ej ->[('update', 1.3937460844372076),
                ('aapl', 0.5944950710074257),
                ('stockaction', 2.0249643045063834)]

  """
  frecuenciasRDD=frecuencias(bagOfWordsRDD)
  # Mapeamos para colocar cada token como valor clave. Ej. -> ('token',('id',frecuencia,1))
  tokenRDD=frecuenciasRDD.map(lambda x: (x[0][1],(x[0][0],x[1],1)))
  # Extraemos el token y su ocurrencia por documento
  tokenOcurrenciaDocRDD=tokenRDD.map(lambda x:(x[0],x[1][2]))
  # contamos la contidad de documentos que contienen un token
  docPerTokenRDD=tokenOcurrenciaDocRDD.reduceByKey(lambda x,y:x+y)
  #calculamos el logaritmo para el IDF
  idfRDD=docPerTokenRDD.map(lambda x: (x[0],math.log10(cantidadDoc/x[1])))
  return idfRDD

In [137]:
#Crear el RDD leyendo el archivo .txt
textRDD=sc.textFile('twitts.txt',4)
datos=preprocesar(textRDD)
cantidadDoc=len(datos)
BoW=bow(datos)
idf=IDF(BoW,cantidadDoc)
idf.take(20)

[('aapl', 0.5944950710074257),
 ('stockaction', 2.0249643045063834),
 ('feedback', 3.104145550554008),
 ('iwatch', 2.627024295834346),
 ('accessory', 2.979206813945708),
 ('turn', 2.1662934573028525),
 ('facetime', 2.4351387695954325),
 ('seamlessly', 3.2802368096096894),
 ('airport', 3.5812668052736707),
 ('board', 2.882296800937652),
 ('security', 2.4351387695954325),
 ('card', 2.3771468226177457),
 ('commute', 3.2802368096096894),
 ('accord', 2.5398741201154458),
 ('ceo', 1.968482948553935),
 ('maker', 2.7361687652594138),
 ('like', 1.5684295805684985),
 ('monarchywpb', 3.5812668052736707),
 ('go', 1.7424177145364153),
 ('pay', 1.781926255820089)]

## **5. TF-IDF**

In [138]:
def TF_IDF(bagOfWordsRDD,cantidadDoc):
  """
  Calcula Inverse document frequency
  
  Parameters
  =========
    `bagOfWordsRDD :` lista de tuplas de tuplas ((idDoc, token),1)
          ej ->[((1, 'protester'), 1),
                ((1, 'stage'), 1),
                ((1, 'diein'), 1)]
    
    `cantidadDoc :` cantidad de documentos (int)
          ej -> 2857
    
  Returns
  =========
    `tfIdfRDD :`  lista de tuplas (idDoc,token,tf, idf,tf-idf)
          ej ->[(1, 'store', 1, 1.105595616949241, 1.105595616949241),
                (1, 'stage', 1, 1.3798696809532192, 1.3798696809532192),
                (1, 'nyc', 1, 1.353380100659997, 1.353380100659997)]

  """
  #obtenemos el RDD con el id del documento y la frecuencia para cada token
  tfRDD=TF(bagOfWordsRDD)
  # obtenemos el RDD con las puntuaciones IDF
  idfRDD=IDF(bagOfWordsRDD,cantidadDoc)
  #combinamos para asignar a cada token un id de documento, frecuencia y puntuación IDF
  tfIdfRDD=tfRDD.join(idfRDD)
  #multiplicamos los valores de TF e IDF de cada token
  tfIdfRDD=tfIdfRDD.map(lambda x: (x[1][0][0],(x[0],x[1][0][1],x[1][1],x[1][0][1]*x[1][1]))).sortByKey()
  #reorganizamos el RDD para tener mejor comprension
  tfIdfRDD=tfIdfRDD.map(lambda x: (x[0],x[1][0],x[1][1],x[1][2],x[1][3]))
  return tfIdfRDD

In [139]:
#Crear el RDD leyendo el archivo .txt
textRDD=sc.textFile('twitts.txt',4)
datos=preprocesar(textRDD)
cantidadDoc=len(datos)
BoW=bow(datos)
tf_idf=TF_IDF(BoW,cantidadDoc)
tf_idf.take(20)

[(1, 'store', 1, 1.105595616949241, 1.105595616949241),
 (1, 'diein', 1, 1.399423217328898, 1.399423217328898),
 (1, 'retweet', 1, 1.399423217328898, 1.399423217328898),
 (1, 'agree', 1, 1.3690792008697128, 1.3690792008697128),
 (1, 'stage', 1, 1.3798696809532192, 1.3798696809532192),
 (1, 'nyc', 1, 1.353380100659997, 1.353380100659997),
 (1, 'protester', 1, 1.388142206919209, 1.388142206919209),
 (1, 'protest', 1, 1.377146822617746, 1.377146822617746),
 (1, 'anger', 1, 1.388142206919209, 1.388142206919209),
 (1, 'misplace', 1, 1.390935107103379, 1.390935107103379),
 (2, 'range', 1, 2.2195389692560776, 2.2195389692560776),
 (2, 'trading', 1, 1.928054291498327, 1.928054291498327),
 (2, 'stock', 1, 1.457415164306585, 1.457415164306585),
 (2, 'mkt', 1, 1.968482948553935, 1.968482948553935),
 (2, 'update', 1, 1.3937460844372076, 1.3937460844372076),
 (2, 'apple', 1, 0.578668824553762, 0.578668824553762),
 (2, 'intraday', 1, 1.6270242958343457, 1.6270242958343457),
 (2, 'premium', 1, 2.0761

## **Resultado**

In [140]:
import pandas as pd
#Creamos el daframe para mostrar los resultados
dfTabla=pd.DataFrame(tf_idf.collect(),columns=['Id Documento','Token','TF','IDF','TF-IDF'])
#mostarlos los 30 primeros resultados obtenidos
dfTabla[:30]

,Id Documento,Token,TF,IDF,TF-IDF
0,1,store,1,1.105596,1.105596
1,1,diein,1,1.399423,1.399423
2,1,retweet,1,1.399423,1.399423
3,1,agree,1,1.369079,1.369079
4,1,stage,1,1.379870,1.379870
5,1,nyc,1,1.353380,1.353380
6,1,protester,1,1.388142,1.388142
7,1,protest,1,1.377147,1.377147
8,1,anger,1,1.388142,1.388142
9,1,misplace,1,1.390935,1.390935
